In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, InjectedToolArg
from langchain_core.messages import HumanMessage
import requests
from typing import Annotated
import json

In [16]:
@tool
def get_conversion_rate(base_currency: str, target_currency:str) -> float:
    """
    This function returns the conversion rate between two currencies
    """
    url = f'https://v6.exchangerate-api.com/v6/fd76e77c9791fd74ea197273/pair/{base_currency}/{target_currency}'

    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency: float, conversion_factor: Annotated[float, InjectedToolArg]) -> float:
    """"
    This function returns the converted amount
    """
    return base_currency * conversion_factor


In [25]:
result = get_conversion_rate.invoke({'base_currency': 'USD', 'target_currency': 'INR'}) 
print(result)
result2 = convert.invoke({'base_currency': 50.2, 'conversion_factor': 89.45})
print(result2)

{'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1764547201, 'time_last_update_utc': 'Mon, 01 Dec 2025 00:00:01 +0000', 'time_next_update_unix': 1764633601, 'time_next_update_utc': 'Tue, 02 Dec 2025 00:00:01 +0000', 'base_code': 'USD', 'target_code': 'INR', 'conversion_rate': 89.4521}
4490.39


In [18]:
llm = ChatOpenAI()
llm_with_tools = llm.bind_tools([get_conversion_rate, convert])


In [28]:
messages = [HumanMessage("Can you tell me what is the ocnversion rate of USD to INR? Also, How much INR would be 50.2 USD?")]
ai_message = llm_with_tools.invoke(messages)
messages.append(ai_message)
print(ai_message)
print(ai_message.content)
print(ai_message.tool_calls)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 110, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Chpe30HOMEfTyUJULeum2Dj48mUif', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--4690562e-42be-488f-8e8e-d4b243b92cb4-0' tool_calls=[{'name': 'get_conversion_rate', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_OG0WZVg5PzydqzmY1Ul2Ho24', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency': 50.2}, 'id': 'call_I0RK6rea2rwNvZrpibOmunPx', 'type': 'tool_call'}] usage_metadata={'input_tokens': 110, 'output_tokens': 53, 'total_tokens': 163, 'input_token_detai

In [31]:
for individual_tool in ai_message.tool_calls:
    if individual_tool['name'] == 'get_conversion_rate':
        tool_msg1 = get_conversion_rate.invoke(individual_tool)
        print(tool_msg1)
        messages.append(tool_msg1)
    if individual_tool['name'] == 'convert':
        conversion_factor = json.loads(tool_msg1.content)['conversion_rate']
        individual_tool['args']['conversion_factor'] = conversion_factor
        tool_msg2 = convert.invoke(individual_tool)
        print(tool_msg2)
        messages.append(tool_msg2)


content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1764547201, "time_last_update_utc": "Mon, 01 Dec 2025 00:00:01 +0000", "time_next_update_unix": 1764633601, "time_next_update_utc": "Tue, 02 Dec 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 89.4521}' name='get_conversion_rate' tool_call_id='call_OG0WZVg5PzydqzmY1Ul2Ho24'
content='4490.49542' name='convert' tool_call_id='call_I0RK6rea2rwNvZrpibOmunPx'


In [32]:
messages

[HumanMessage(content='Can you tell me what is the ocnversion rate of USD to INR? Also, How much INR would be 50.2 USD?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 110, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Chpe30HOMEfTyUJULeum2Dj48mUif', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4690562e-42be-488f-8e8e-d4b243b92cb4-0', tool_calls=[{'name': 'get_conversion_rate', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_OG0WZVg5PzydqzmY1Ul2Ho24', 'type': 'tool_call'}, {'name': 'convert', 'args': {

In [33]:
final_result = llm_with_tools.invoke(messages)
print(final_result)

content='The conversion rate from USD to INR is 89.4521. \n\nIf you have 50.2 USD, it would be equivalent to approximately 4490.50 INR.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 324, 'total_tokens': 364, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Chpj6kgwfr78Gimh3uGJqCjA3rQmL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--85786b84-8260-4bd1-ae1c-b3d7dd00c9fc-0' usage_metadata={'input_tokens': 324, 'output_tokens': 40, 'total_tokens': 364, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
